[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jvdzwaan/ocrpostcorrection/blob/main/colab/icdar-task1-hf-evaluation.ipynb)

In [ ]:
from google.colab import drive 
drive.mount('/mntDrive')

Mounted at /mntDrive


In [ ]:
!git clone https://github.com/jvdzwaan/ocrpostcorrection.git

Cloning into 'ocrpostcorrection'...
remote: Enumerating objects: 471, done.
remote: Counting objects: 100% (471/471), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 471 (delta 265), reused 399 (delta 196), pack-reused 0
Receiving objects: 100% (471/471), 904.27 KiB | 10.16 MiB/s, done.
Resolving deltas: 100% (265/265), done.


In [ ]:
!pip install ./ocrpostcorrection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ocrpostcorrection
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 365 kB 7.5 MB/s 
     |████████████████████████████████| 305 kB 60.2 MB/s 
     |████████████████████████████████| 58 kB 7.5 MB/s 
     |████████████████████████████████| 4.9 MB 57.9 MB/s 
     |████████████████████████████████| 212 kB 73.7 MB/s 
     |████████████████████████████████| 120 kB 74.9 MB/s 
     |████████████████████████████████| 115 kB 76.2 MB/s 
     |████████████████████████████████| 127 kB 75.6 MB/s 
     |███████

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_from_disk

icdar_dataset = load_from_disk('/mntDrive/MyDrive/icdar-seq_len-150')
#icdar_dataset = load_from_disk('../../data/ocrpostcorrection/icdar-seq_len-150')

In [ ]:
for split in icdar_dataset.keys():
    icdar_dataset[split] = icdar_dataset[split].select(range(5))

In [ ]:
model_dir = '/mntDrive/MyDrive/results-seq_len-150-0.3'
#model_dir = '/Users/janneke/models/results-seq_len-150-0.3'
model_name = 'bert-base-multilingual-cased'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
from ocrpostcorrection.token_classification import tokenize_and_align_labels

tokenized_icdar = icdar_dataset.map(tokenize_and_align_labels(tokenizer), batched=True)

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=model_dir,          # output directory
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

model = AutoModelForTokenClassification.from_pretrained(model_dir, num_labels=3)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_icdar['train'],         # training dataset
    eval_dataset=tokenized_icdar['val'],            # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
pred = trainer.predict(tokenized_icdar['test'])

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: start_token_id, key, tokens, tags, score, language. If start_token_id, key, tokens, tags, score, language are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7010
  Batch size = 8
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
pred.metrics

{'test_loss': 0.4178357720375061,
 'test_runtime': 106.9982,
 'test_samples_per_second': 65.515,
 'test_steps_per_second': 8.196}

In [ ]:
from pathlib import Path

out_dir = Path('/mntDrive/MyDrive/results/icdar-seq_len-150')
out_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
# Save predictions
import numpy as np

out_file = out_dir/'predictions'

np.save(out_file, pred.predictions)

# Can be run locally

In [ ]:
# Load predictions
import numpy as np

in_file = out_dir/'predictions.npy'

predictions = np.load(in_file)

In [ ]:
from ocrpostcorrection.icdar_data import generate_data

in_dir = Path('../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish')
data_test, X_test = generate_data(in_dir)

5it [00:03,  1.34it/s]2022-09-16 20:11:40.254 | WARNING  | ocrpostcorrection.icdar_data:process_text:185 - UnicodeEncodeError for text ../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/BG/BG1/47.txt; setting score to 1
11it [00:30,  2.74s/it]


In [ ]:
from ocrpostcorrection.utils import predictions2icdar_output, predictions_to_labels

output = predictions2icdar_output(tokenized_icdar['test'],
                                  predictions_to_labels(predictions),
                                  tokenizer,
                                  data_test)

In [ ]:
import json

out_file = out_dir/'results_task1_unfiltered_test_data.json'

with open(out_file, 'w') as f:
    json.dump(output, f)

In [ ]:
csv_file = out_file.with_suffix('.csv')

In [ ]:
from ocrpostcorrection.utils import runEvaluation

runEvaluation(in_dir, out_file, csv_file)

File	NbTokens	NbErroneousTokens	NbSymbolsConsidered	T1_Precision	T1_Recall	T1_Fmesure	T2_AvgLVDistOriginal	T2_AvgLVDistCorrected
SL/SL1/29.txt	2	2	533	1.00	0.74	0.85	0.99	0.99
SL/SL1/15.txt	83	28	447	0.45	0.90	0.60	0.14	0.14
SL/SL1/14.txt	115	50	721	0.45	0.84	0.59	0.13	0.13
SL/SL1/28.txt	273	21	1453	0.12	0.94	0.21	0.03	0.03


In [ ]:
from ocrpostcorrection.utils import aggregate_results

results = aggregate_results(csv_file)
print(results.to_markdown())

| language   |   T1_Precision |   T1_Recall |   T1_Fmesure |
|:-----------|---------------:|------------:|-------------:|
| SL         |          0.505 |       0.855 |       0.5625 |
